In [1]:
%load_ext autoreload
%autoreload 2
directory = '/scratch/6256481/'
import warnings
warnings.filterwarnings("ignore")

import glob
#%% Import packages
import glob
import os
from datetime import datetime as dt
from tqdm import tqdm 
import traceback

import geopandas as gpd
import pandas as pd
import numpy as np
from multiprocessing import Pool
# import scipy as sc

import logging

# import shapely
from shapely.geometry import LineString 


# os.chdir(directory + 'py_code')
os.chdir(directory + 'python/py_code')
# print(os.getcwd())

# from tqdm import tqdm




# %% Import functions
from remove_mannual_add import remove_manual_add, remove_man_add
from smoothing import SG_smoothing
from reach_def import create_ud_id, create_new_segments


from width_ratio import width_ratio

from valley_line import river_dir, inflection_valley
from slope_confinemend import valley_slope_and_confinemend
from remove import remove_dems

from dem import find_dem
from get_orthogonals import get_orthogonals

from cycle_identification import find_connected_side
from support import create_dir,  get_local_utm_projection


In [12]:
F = glob.glob(directory + 'input/SWOT_nodes/*gpkg')

F = np.sort(glob.glob(directory + f'results/new_segments/vector/??_??_*.gpkg'))
F[0][-29:-27]
F[0][-26:-24]

'00'

In [116]:
c = 'af'
i = 11
print(f'Start: {c}_{i}. Time: {dt.now()}')
code_failure = 0



projection       = 'EPSG:3857'
projection_coord = 'EPSG:4326'


def create_new_reach_files(directory, c, i):
    vector_file = glob.glob(directory + f'input/SWOT_vector/{c}*{i}_*shp')[0]
    node_file   = glob.glob(directory + f'input/SWOT_nodes/{c}*{i}_*shp')[0]

    df_vector       = gpd.read_file(vector_file)
    if df_vector.crs is None:
        df_vector = df_vector.set_crs(projection_coord)

    df_vector = df_vector.to_crs(projection)
    df_vector_whole = df_vector.copy()

    # add local projection to dataframe
    df_vector_whole = get_local_utm_projection(df_vector_whole)
    
    df_vector_whole = df_vector_whole[df_vector_whole['type'] != 6]      # exclude ghostreaches
    df_vector_whole = df_vector_whole[df_vector_whole.reach_len > (500)] # minimum reachLength


    # finds connected reaches and creates cycles
    cycleFile = directory + f'results/cycles/{c}_{i}_cycle.shp'
    df_cycle, dfConnectionNodes  = find_connected_side(df_vector_whole, projection, cycleFile)
    
    
    df_vector_whole = pd.merge(df_vector_whole, df_cycle, how = 'left', on = 'reach_id')
    df_vector       = df_vector_whole[(df_vector_whole['type'] == 1) & (df_vector_whole['lakeflag'] == 0)]
    
    df_node = gpd.read_file(node_file)
    if df_node.crs is None:
        df_node = df_node.set_crs(projection)
    else:
        df_node = df_node.to_crs(projection)
    
    
    df_node = df_node[df_node.reach_id.isin(df_vector.reach_id)]
    

    # remove parts of segments with manually added nodes
    df_vector, df_node_nm = remove_man_add(df_vector,df_node)

    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # CHANGE TO CORRECT LENGTH CHECK!!!!!!!!!

    # create new segments from upstream and downstream nodes
    df_upstream     = create_ud_id(df_vector,df_vector_whole,df_node, df_node_nm)
    
    df_new_segments = create_new_segments(df_upstream, df_upstream, df_node, projection)

    df = df_new_segments.copy()

    # width ratio
    df = width_ratio(df, df_node)

    # Save files
    df.to_file(vector_save_file      
                , driver='ESRI Shapefile', crs = projection)
    df_node.to_file(node_save_file      
                , driver='ESRI Shapefile', crs = projection)
    
    return df, df_node
create_new = True
calc       = True


vector_save_file = directory +f'results/new_segments/vector/{c}_{i}_new_segments_vector.shp'
node_save_file   = directory +f'results/new_segments/node/{c}_{i}_new_segments_node.shp' 

df, df_node = create_new_reach_files(directory, c, i)


df[['reach_id', 'reach_len', 'LCheck']][abs(df.reach_len - df.LCheck) > 2e3]


Start: af_11. Time: 2024-10-07 18:39:50.916378


,reach_id,reach_len,LCheck


In [103]:
df2 = gpd.read_file(np.sort(glob.glob(directory + 'results/new_segments/vector/*.shp'))[0])
df2.head(10)

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,az_dn,order,LCheck,rch_ids_up,rch_ids_dn,node_mwm,min_len,short,widthRatio,geometry
0,42.582381,-0.135846,11410000021,10495.153532,52,1.7,0.000000,94.0,495.646181,740052.747031,...,NaN,0,10495.153532,None,None,99.423077,4772.307692,0,1.0,"LINESTRING (4742486.190 -18852.208, 4742480.19..."
1,42.594848,-0.091447,11410000031,10442.226833,52,1.7,0.030291,84.0,230.260156,739619.603428,...,40.292887,0,10442.226833,None,None,85.019231,4080.923077,0,1.0,"LINESTRING (4740663.715 -12518.670, 4740693.70..."
2,42.608105,-0.040976,11430000041,11266.981232,56,5.2,1.325533,77.0,438.047619,739160.208697,...,12.884197,0,11266.981232,None,None,77.187500,3705.000000,0,1.0,"LINESTRING (4740903.933 -7410.988, 4740891.940..."
3,42.654059,-0.021237,11430000051,10936.059420,55,7.4,0.065843,63.0,300.498249,738668.729254,...,95.573306,0,10936.059420,None,None,61.045455,2930.181818,0,1.0,"LINESTRING (4746331.519 -3686.002, 4746361.506..."
4,42.706236,0.037553,11430000011,18851.017314,94,7.7,0.000000,63.0,441.705080,510965.062500,...,28.557706,0,18851.017314,None,None,59.728723,2866.978723,0,1.0,"LINESTRING (4748910.418 0.002, 4748910.418 15...."
5,42.718510,0.104512,11430000021,18801.655579,94,10.6,1.028028,63.0,406.029954,475345.548832,...,40.764646,0,18801.655579,None,None,64.617021,3101.617021,0,1.0,"LINESTRING (4756107.259 7834.385, 4756077.271 ..."
6,42.748166,0.176689,11430000031,18866.814329,94,12.3,0.525542,62.0,646.252508,439536.569665,...,43.465142,0,18866.814329,None,None,59.196809,2841.446809,0,1.0,"LINESTRING (4755746.800 15961.590, 4755716.809..."
7,42.759419,0.259603,11441100011,17572.319716,87,15.9,0.285256,63.0,559.068599,211597.031250,...,10.715866,0,17572.319716,None,None,62.214080,2986.275862,0,1.0,"LINESTRING (4761353.633 24741.813, 4761343.630..."
8,42.759847,0.312713,11441100021,10496.452471,52,17.9,0.461576,63.0,440.060661,210987.981142,...,37.041098,0,10496.452471,None,None,64.653846,3103.384615,0,1.0,"LINESTRING (4759463.088 31794.193, 4759493.074..."
9,42.750952,0.372144,11441100031,10493.710481,52,17.9,0.899321,67.0,377.929939,210626.759484,...,29.620819,0,10493.710481,None,None,67.894231,3258.923077,0,1.0,"LINESTRING (4760895.325 37754.302, 4760877.324..."


In [117]:
df3 = gpd.read_file(np.sort(glob.glob(directory + 'results/new_segments/vector/*.shp'))[0])

In [125]:
df3[df3.reach_id ==11744700021 ]
df2[df2.reach_id ==11744700021 ]

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,az_dn,order,LCheck,rch_ids_up,rch_ids_dn,node_mwm,min_len,short,widthRatio,geometry
869,36.579792,-8.244584,11744700021,15725.607672,7,245.300003,0.0,129.0,5453.36,29671.027344,...,150.81177,2,13529.222565,None,11744600141 11744600051,550.5,26424.0,1,0.581411,"LINESTRING (4072052.651 -920295.734, 4072052.6..."


In [126]:
len(df3[~df3.rch_ids_dn.isna()].reach_id.values)
len(df2[~df2.rch_ids_dn.isna()].reach_id.values)

162

In [115]:
t = df2[~df2.rch_ids_up.isna()].reach_id.values
df2[df2.reach_id.isin(t)]

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,az_dn,order,LCheck,rch_ids_up,rch_ids_dn,node_mwm,min_len,short,widthRatio,geometry
63,42.404808,4.026535,11443000011,35734.547732,44,158.500000,0.383185,189.0,1183.728944,142379.156250,...,23.967303,1,35704.547580,11443000021,11443000561,230.579545,11067.818182,0,0.932283,"LINESTRING (4710503.758 453018.282, 4710479.77..."
65,42.310045,4.089081,11443000031,34856.255546,44,163.300003,1.974269,183.0,1984.207434,142068.870030,...,32.286115,1,34795.661317,11443000041,11443000021,220.613636,10589.454545,0,0.932756,"LINESTRING (4698446.959 463245.597, 4698470.91..."
115,41.993532,4.306487,11447000061,20167.423392,18,182.800003,0.314453,94.0,629.172840,57168.714844,...,0.294268,1,20105.378737,11447000051,11445000081,95.138889,4566.666667,0,1.000000,"LINESTRING (4675429.190 478269.252, 4675459.31..."
254,41.999164,7.927082,11496000021,17947.412883,7,601.200012,0.000000,36.0,188.066107,9502.773438,...,35.546510,1,17880.584165,11496000041,11496000711,32.517857,1560.857143,0,1.000000,"LINESTRING (4675423.262 886966.342, 4675447.68..."
279,36.121948,3.091569,11542400011,20637.088605,51,372.100006,21.121980,123.0,4841.127387,22788.937500,...,NaN,1,20598.669704,11542400021,None,343.745098,16499.764706,0,0.715598,"LINESTRING (4007505.703 346111.053, 4007535.75..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,37.008496,-16.717280,11796000121,40595.945680,52,123.900002,17.143475,89.5,2393.466462,12318.996064,...,63.852870,1,40531.323681,11796000131,11796000111,227.307692,10910.769231,0,0.648414,"LINESTRING (4124723.500 -1876017.115, 4124691...."
1188,37.111184,-16.466053,11796000161,32114.876574,53,203.900009,10.318030,112.0,1030.736114,9868.683228,...,39.118653,1,32054.876507,11796000171,11796000151,343.141509,16470.792453,0,0.676983,"LINESTRING (4133403.451 -1845612.402, 4133421...."
1189,37.120551,-16.393865,11796000171,32141.508942,53,238.100006,199.208182,90.0,1806.574277,9243.030228,...,1.920027,1,32076.523208,11796000181,11796000161,339.754717,16308.226415,0,0.606816,"LINESTRING (4127232.516 -1837756.746, 4127250...."
1190,37.103639,-16.315151,11796000181,32108.262801,53,260.300018,15.158777,105.0,786.913369,8607.692196,...,15.156794,1,32038.291332,11796000191,11796000171,335.245283,16091.773585,0,0.635031,"LINESTRING (4132074.536 -1828944.595, 4132043...."


In [114]:
df[df.reach_id.isin(t)]


,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,az_up,az_dn,order,LCheck,rch_ids_up,rch_ids_dn,node_mwm,min_len,short,widthRatio
63,42.404808,4.026535,11443000011,35734.547732,44,158.500000,0.383185,189.0,1183.728944,142379.156250,...,9.107383,23.967303,1,35704.547580,11443000021,11443000561,230.579545,11067.818182,False,0.932283
65,42.310045,4.089081,11443000031,34856.255546,44,163.300003,1.974269,183.0,1984.207434,142068.870030,...,8.265359,32.286115,1,34795.661317,11443000041,11443000021,220.613636,10589.454545,False,0.932756
115,41.993532,4.306487,11447000061,20167.423392,18,182.800003,0.314453,94.0,629.172840,57168.714844,...,13.100527,0.294268,1,20105.378737,11447000051,11445000081,95.138889,4566.666667,False,1.000000
254,41.999164,7.927082,11496000021,17947.412883,7,601.200012,0.000000,36.0,188.066107,9502.773438,...,24.379108,35.546510,1,17880.584165,11496000041,11496000711,32.517857,1560.857143,False,1.000000
279,36.121948,3.091569,11542400011,20637.088605,51,372.100006,21.121980,123.0,4841.127387,22788.937500,...,17.157025,NaN,1,20598.669704,11542400021,,343.745098,16499.764706,False,0.715598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,37.008496,-16.717280,11796000121,40595.945680,52,123.900002,17.143475,89.5,2393.466462,12318.996064,...,19.746589,63.852870,1,40531.323681,11796000131,11796000111,227.307692,10910.769231,False,0.648414
1188,37.111184,-16.466053,11796000161,32114.876574,53,203.900009,10.318030,112.0,1030.736114,9868.683228,...,1.920027,39.118653,1,32054.876507,11796000171,11796000151,343.141509,16470.792453,False,0.676983
1189,37.120551,-16.393865,11796000171,32141.508942,53,238.100006,199.208182,90.0,1806.574277,9243.030228,...,15.156794,1.920027,1,32076.523208,11796000181,11796000161,339.754717,16308.226415,False,0.606816
1190,37.103639,-16.315151,11796000181,32108.262801,53,260.300018,15.158777,105.0,786.913369,8607.692196,...,11.344690,15.156794,1,32038.291332,11796000191,11796000171,335.245283,16091.773585,False,0.635031


In [64]:
A = df[df.reach_id == 11744700021]

a = A.iloc[0]
midString = a[['Mid', 'Mid1', 'Mid2']].values
mids = []
for i in midString:
    if i != 'nan':
        mids.extend(eval(i))
mids


[]

In [88]:
a[['reach_id', 'single_up', 'single_dn', 'Value', 'rch_id_up']]

reach_id                    11744700021
single_up                          <NA>
single_dn                   11744600141
Value        [11744600141, 11744500091]
rch_id_up                          None
Name: 869, dtype: object

In [96]:
V = eval(a.Value)

def check_mid_connecting_reach(rid:'str', rch_id : 'str', df:'gpd.GeoDataFrame'):
    checkRow = df[df.reach_id == rch_id]

    midString = checkRow[['Mid', 'Mid1', 'Mid2']].values[0]
    for m in midString:
        if m != 'nan':
            mid = eval(m)
            if mid == rid:
                rch_id = np.nan

    return rch_id
            
check_mid_connecting_reach(a.reach_id, a['single_dn'], df)

nan